#1.Installation de Pandas Profiling

In [1]:
import sys
!{sys.executable} -m pip install -U ydata-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.3/357.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.5 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=1d9cbca613dfe79bbdd33e00d3e8507b799e8e06cdecc2dae49d7578d546213a
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.2.1
    Uninstalling pydantic-2.2.

#2.Chargement des librairies

In [2]:
# System
import os
from google.colab import files
import warnings

# Data
import numpy as np
import pandas as pd
import json
import datetime
from sqlalchemy import create_engine, text
from ydata_profiling import ProfileReport

# Machine learning - Preprocessing
from sklearn.impute import KNNImputer

# Graphics
import plotly.express as px

#3.Configuration

In [3]:
# Silence warnings
warnings.filterwarnings("ignore")

In [4]:
# Mount GoogleDrive and set the files path
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/CO2'

Mounted at /content/drive
/content/drive/MyDrive/CO2


#4.Chargement du dataset

In [5]:
# Define the connection string
connection_string = "postgresql://co2-sa-db.postgres.database.azure.com:5432/seattlebeb?user=co2sodapg&password=Greta2023&sslmode=require"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Create a connection and execute the query
with engine.connect() as conn:
    query = text("SELECT * FROM buildings")
    data = pd.read_sql(query, conn)

data.head()

,osebuildingid,buildingtype,primarypropertytype,zipcode,taxparcelidentificationnumber,councildistrictcode,neighborhood,latitude,longitude,yearbuilt,...,sourceeuiwn_kbtu_sf,siteenergyuse_kbtu,siteenergyusewn_kbtu,steamuse_kbtu,electricity_kbtu,naturalgas_kbtu,defaultdata,compliancestatus,outlier,totalghgemissions
0,1,NonResidential,Hotel,98101.0,0659000030,7,DOWNTOWN,47.61220,-122.33799,1927,...,189.000000,7226362.5,7456910.0,2003882.00,3946027.0,1276453.0,False,Compliant,None,249.98
1,2,NonResidential,Hotel,98101.0,0659000220,7,DOWNTOWN,47.61317,-122.33393,1996,...,179.399994,8387933.0,8664479.0,0.00,3242851.0,5145082.0,False,Compliant,None,295.86
2,3,NonResidential,Hotel,98101.0,0659000475,7,DOWNTOWN,47.61393,-122.33810,1969,...,244.100006,72587024.0,73937112.0,21566554.00,49526664.0,1493800.0,False,Compliant,None,2089.28
3,5,NonResidential,Hotel,98101.0,0659000640,7,DOWNTOWN,47.61412,-122.33664,1926,...,224.000000,6794584.0,6946800.5,2214446.25,2768924.0,1811213.0,False,Compliant,None,286.43
4,8,NonResidential,Hotel,98121.0,0659000970,7,DOWNTOWN,47.61375,-122.34047,1980,...,215.600006,14172606.0,14656503.0,0.00,5368607.0,8803998.0,False,Compliant,None,505.01


#5.Nettoyage des données

In [6]:
# Remove IDs
df = data.drop(['osebuildingid', 'taxparcelidentificationnumber'], axis=1)

Les données de l'année 2019 contiennent une colonne supplémentaire, 'complianceissue', qui fait mieux comprendre à quoi correspond la feature 'compliancestatus' : Account Requires Verification, Missing 2019 EUI or Electricity Data, Portfolio Manager Account Not Shared, Default Data...
https://www.opendatanetwork.com/dataset/data.seattle.gov/3th6-ticf

Pour obtenir une modélisation plus fiable, nous pouvons donc supprimer toutes les lignes correspondant aux bâtiments ayant un statut autre que 'Compliant'. Cela nous permettra également de supprimer la colonne 'defaultdata'.

In [7]:
df = df[df['compliancestatus'] == 'Compliant']
df['defaultdata'].value_counts()

False    3211
Name: defaultdata, dtype: int64

In [8]:
df.drop(['defaultdata', 'compliancestatus'], axis=1, inplace=True)

In [9]:
# Drop all rows where the 'outlier' column is not null
df.drop(index=df[df['outlier'].notna()].index, inplace=True)
df.drop(['outlier'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3211 entries, 0 to 3375
Data columns (total 32 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   buildingtype                     3211 non-null   object 
 1   primarypropertytype              3211 non-null   object 
 2   zipcode                          3198 non-null   float64
 3   councildistrictcode              3211 non-null   int64  
 4   neighborhood                     3211 non-null   object 
 5   latitude                         3211 non-null   float64
 6   longitude                        3211 non-null   float64
 7   yearbuilt                        3211 non-null   int64  
 8   numberofbuildings                3208 non-null   float64
 9   numberoffloors                   3211 non-null   int64  
 10  propertygfatotal                 3211 non-null   int64  
 11  propertygfaparking               3211 non-null   int64  
 12  propertygfabuilding_

In [10]:
# Impute 'zipcode' missing values from 'latitude' and 'longitude'
location = ['zipcode', 'latitude', 'longitude']
dfl = df[location]
dfl = pd.DataFrame(data=KNNImputer(n_neighbors=10).fit_transform(dfl), index=df.index, columns=location)
df = df.drop(columns=location).join(dfl)

In [11]:
df['neighborhood'].value_counts()

DOWNTOWN                  556
EAST                      440
MAGNOLIA / QUEEN ANNE     412
GREATER DUWAMISH          357
NORTHEAST                 266
LAKE UNION                244
NORTHWEST                 198
SOUTHWEST                 149
NORTH                     136
BALLARD                   120
CENTRAL                    95
SOUTHEAST                  78
DELRIDGE                   73
North                      40
Central                    26
Northwest                  10
Ballard                     6
Delridge                    4
DELRIDGE NEIGHBORHOODS      1
Name: neighborhood, dtype: int64

In [12]:
# Fix the case and replace duplicates
df['neighborhood'] = df['neighborhood'].str.upper()
df['neighborhood'] = df['neighborhood'].str.replace('DELRIDGE NEIGHBORHOODS','DELRIDGE')
df['neighborhood'].value_counts()

DOWNTOWN                 556
EAST                     440
MAGNOLIA / QUEEN ANNE    412
GREATER DUWAMISH         357
NORTHEAST                266
LAKE UNION               244
NORTHWEST                208
NORTH                    176
SOUTHWEST                149
BALLARD                  126
CENTRAL                  121
SOUTHEAST                 78
DELRIDGE                  78
Name: neighborhood, dtype: int64

In [13]:
# Fix 'energystarscore'
df['energystarscore'] = df['energystarscore'].replace("NULL", None)
df['energystarscore'] = df['energystarscore'].astype('object')

In [14]:
# Drop columns with too many missing values
threshold = 70 # minimum percentage of non-null cells in each column
for column in df.columns:
  if df[column].isnull().sum() / len(df) *100 > (100 - threshold):
      df.drop([column], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3211 entries, 0 to 3375
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   buildingtype               3211 non-null   object 
 1   primarypropertytype        3211 non-null   object 
 2   councildistrictcode        3211 non-null   int64  
 3   neighborhood               3211 non-null   object 
 4   yearbuilt                  3211 non-null   int64  
 5   numberofbuildings          3208 non-null   float64
 6   numberoffloors             3211 non-null   int64  
 7   propertygfatotal           3211 non-null   int64  
 8   propertygfaparking         3211 non-null   int64  
 9   propertygfabuilding_s      3211 non-null   int64  
 10  listofallpropertyusetypes  3207 non-null   object 
 11  largestpropertyusetype     3207 non-null   object 
 12  largestpropertyusetypegfa  3196 non-null   float64
 13  energystarscore            2399 non-null   objec

In [15]:
# Remove '_s' to help retrieving feature names after the OneHotEncoder
df.rename(columns={'propertygfabuilding_s': 'propertygfabuilding'}, inplace=True)

In [16]:
# Change the feature types
df['zipcode'] = df['zipcode'].astype('object')
df['councildistrictcode'] = df['councildistrictcode'].astype('object')
df['numberofbuildings'] = df['numberofbuildings'].astype('Int64')
df['largestpropertyusetypegfa'] = df['largestpropertyusetypegfa'].astype('Int64')
df.dtypes

buildingtype                  object
primarypropertytype           object
councildistrictcode           object
neighborhood                  object
yearbuilt                      int64
numberofbuildings              Int64
numberoffloors                 int64
propertygfatotal               int64
propertygfaparking             int64
propertygfabuilding            int64
listofallpropertyusetypes     object
largestpropertyusetype        object
largestpropertyusetypegfa      Int64
energystarscore               object
siteeui_kbtu_sf              float64
siteeuiwn_kbtu_sf            float64
sourceeui_kbtu_sf            float64
sourceeuiwn_kbtu_sf          float64
siteenergyuse_kbtu           float64
siteenergyusewn_kbtu         float64
steamuse_kbtu                float64
electricity_kbtu             float64
naturalgas_kbtu              float64
totalghgemissions            float64
zipcode                       object
latitude                     float64
longitude                    float64
d

#6.Feature Engineering

Pour faciliter la modélisation, nous allons calculer l'âge des bâtiments à partir de l'année de construction :

In [17]:
current_year = datetime.datetime.now().year
df['age'] = df['yearbuilt'].apply(lambda x: current_year - x)
df.drop(['yearbuilt'], axis=1, inplace=True)

Le brief indique que "les relevés sont coûteux à obtenir". Pour les remplacer, nous avons tenté un encodage ordinal qui note chaque bâtiment en fonction du type d'énergie consommée ou générée :

2 si le bâtiment est générateur net

1 si son solde est nul

0 s'il est consommateur net

Mais les données 2016 ne sont pas suffisamment bien distribuées pour que cette approche soit concluante. Au final, c'est un traitement catégorique de type OneHot qui donne les meilleurs résultats :

In [18]:
def energy_usage(cell):
    if cell > 0:
      return 'Yes'
    else:
      return 'No'
df['steamuse_kbtu'] = df['steamuse_kbtu'].apply(energy_usage)
df['electricity_kbtu'] = df['electricity_kbtu'].apply(energy_usage)
df['naturalgas_kbtu'] = df['naturalgas_kbtu'].apply(energy_usage)
df.rename(columns={'steamuse_kbtu': 'steam'}, inplace=True)
df.rename(columns={'electricity_kbtu': 'electricity'}, inplace=True)
df.rename(columns={'naturalgas_kbtu': 'naturalgas'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3211 entries, 0 to 3375
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   buildingtype               3211 non-null   object 
 1   primarypropertytype        3211 non-null   object 
 2   councildistrictcode        3211 non-null   object 
 3   neighborhood               3211 non-null   object 
 4   numberofbuildings          3208 non-null   Int64  
 5   numberoffloors             3211 non-null   int64  
 6   propertygfatotal           3211 non-null   int64  
 7   propertygfaparking         3211 non-null   int64  
 8   propertygfabuilding        3211 non-null   int64  
 9   listofallpropertyusetypes  3207 non-null   object 
 10  largestpropertyusetype     3207 non-null   object 
 11  largestpropertyusetypegfa  3196 non-null   Int64  
 12  energystarscore            2399 non-null   object 
 13  siteeui_kbtu_sf            3211 non-null   float

#7.Target Engineering

Scikit-learn propose deux solutions pour gérer les targets multiples :
- MultiOutputRegressor si les variables sont traitées de façon indépendante.
- RegressorChain si elles sont dépendantes.

https://scikit-learn.org/stable/modules/multiclass.html

Il y a une corrélation élevée entre 'siteenergyuse_kbtu' et 'totalghgemissions', donc nous choisirons la seconde option. Mais nous pouvons aller plus loin en incluant dans les targets toutes les variables relatives à la consommation énergétique : il est possible que le modèle s'améliore progressivement en cherchant à les prédire.

Nous allons également créer une variable qui va décrire un bâtiment en fonction du ratio source/site en valeur normalisée :

In [19]:
df['source_site'] = df['sourceeuiwn_kbtu_sf'] / df['siteeuiwn_kbtu_sf']
df['source_site'].describe()

count    3195.000000
mean        2.560808
std         0.629469
min        -0.420000
25%         2.121451
50%         2.649485
75%         3.138501
max        13.211111
Name: source_site, dtype: float64

Les deux variables suivantes vont exposer les variations météorologiques afin que le modèle puisse plus facilement inférer 'siteenergyuse_kbtu' à partir de  'siteenergyusewn_kbtu'.

In [20]:
df['source_wn'] = df['sourceeuiwn_kbtu_sf'] / df['sourceeui_kbtu_sf']
df['source_wn'].describe()

count    3201.000000
mean        1.038982
std         0.081679
min        -1.050000
25%         1.014719
50%         1.042604
75%         1.066225
max         1.206434
Name: source_wn, dtype: float64

In [21]:
df['site_wn'] = df['siteeuiwn_kbtu_sf'] / df['siteeui_kbtu_sf']
df['site_wn'].describe()

count    3210.000000
mean        1.051669
std         0.075259
min         0.000000
25%         1.025888
50%         1.051652
75%         1.076923
max         1.275000
Name: site_wn, dtype: float64

Maintenant que ces deux targets ont été créées, nous pouvons nous passer des valeurs non normalisées :

In [22]:
df.drop(['sourceeui_kbtu_sf', 'siteeui_kbtu_sf'], axis=1, inplace=True)

Il reste à déterminer la chronologie des targets dans le RegressorChain.

 Les émissions de CO2 constituent la variable la plus difficile à déterminer, donc la target 'totalghgemissions' sera placée en dernier. Juste avant, nous aurons l'autre variable que nous souhaitons réellement prédire, 'siteenergyuse_kbtu', précédée par sa petite sœur, 'siteenergyusewn_kbtu', qui devrait être légèrement plus facile parce que le lissage dans le temps lui permet d'échapper aux aléas climatiques.

Il vaut mieux placer les targets 'source_wn', 'site_wn' et 'source_site' après les variables dont elles sont dérivées, afin qu'elles bénéficient d'une prédiction de base.

In [23]:
targets = ['sourceeuiwn_kbtu_sf', 'source_wn', 'siteeuiwn_kbtu_sf', 'site_wn', 'source_site', 'siteenergyusewn_kbtu', 'siteenergyuse_kbtu', 'totalghgemissions']

Dans la mesure où il n'y a pas de catégorie spécifique pour les bâtiments qui sont générateurs d'énergie, les valeurs négatives peuvent être ramenées à zéro, ce qui permettra d'appliquer une transformation log si nécessaire.

In [24]:
def transform_negatives(cell):
    if cell < 0:
      return 0
    else:
      return cell
for target in targets:
    df[target] = df[target].apply(transform_negatives)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3211 entries, 0 to 3375
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   buildingtype               3211 non-null   object 
 1   primarypropertytype        3211 non-null   object 
 2   councildistrictcode        3211 non-null   object 
 3   neighborhood               3211 non-null   object 
 4   numberofbuildings          3208 non-null   Int64  
 5   numberoffloors             3211 non-null   int64  
 6   propertygfatotal           3211 non-null   int64  
 7   propertygfaparking         3211 non-null   int64  
 8   propertygfabuilding        3211 non-null   int64  
 9   listofallpropertyusetypes  3207 non-null   object 
 10  largestpropertyusetype     3207 non-null   object 
 11  largestpropertyusetypegfa  3196 non-null   Int64  
 12  energystarscore            2399 non-null   object 
 13  siteeuiwn_kbtu_sf          3210 non-null   float

In [25]:
# Remove the few rows where target values are missing
df.dropna(subset=['sourceeuiwn_kbtu_sf'], inplace=True)
df.dropna(subset=['source_wn'], inplace=True)
df.dropna(subset=['siteeuiwn_kbtu_sf'], inplace=True)
df.dropna(subset=['site_wn'], inplace=True)
df.dropna(subset=['source_site'], inplace=True)
df.dropna(subset=['siteenergyusewn_kbtu'], inplace=True)
df.dropna(subset=['siteenergyuse_kbtu'], inplace=True)
df.dropna(subset=['totalghgemissions'], inplace=True)

#8.Data Profiling

In [26]:
# Fix for the Google Colab bug "ValueError: Only supported for TrueType fonts"
# The report gets generated externally by the profiling.py file
df.to_csv('co2_eda.csv', index=False)
files.download('co2_eda.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

L'intégralité du rapport est disponible sur https://seabeb.azurewebsites.net/dashboard (ou http://127.0.0.1:8000/dashboard si la web app est arrêtée).

La rubrique "Interactions" montre que les émissions de CO2 tendent à croître avec le score ENERGY STAR, ce qui représente un résultat surprenant. Nous verrons dans la partie modélisation si ce score peut nous aider à obtenir de bonnes prédictions.

Les émissions semblent concentrées dans le District 7, qui correspond à Downtown et South Lake Union. Mais une analyse par neighborhood permet de détecter des zones où le niveau de pollution est encore plus problématique (cf. graphique de la section 10).

#9.Sélection des features

Idéalement, cette tâche pourrait être effectuée automatiquement avec [RFECV](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html), mais cet algorithme requiert des ressources très élevées et la tentative effectuée avec LightGBM offre le strict minimum : en ne conservant que les features recommandées, on obtient un score légèrement moins bon (.tests/lightgbm_rfecv.ipynb).

A la place, nous allons effectuer une première sélection sur des bases staistiques, puis XGBoost et sa régularisation L1 nous aideront à progresser.

La variable 'electricity' peut être négligée parce qu'elle est fortement corrélée à la nouvelle feature 'source_wn' et présente un grand déséquilibre.

La variable 'primarypropertytype' sera préférée à 'buildingtype' parce qu'elle est davantage corrélée avec les targets. Pour la même raison, nous choisirons 'propertygfatotal' à la place de 'propertygfabuilding' ou 'largestpropertyusetypegfa'.

Une tentative de géo-clustering (.tests/geo_clusters.ipynb) semble indiquer que la localisation ne représente pas un facteur majeur dans la prédiction. Parmi toutes les variables de ce type qui sont plus ou moins corrélées, 'councildistrictcode' apparaît comme la plus utile.

In [27]:
df_stats = df.drop(['electricity', 'buildingtype', 'propertygfabuilding', 'largestpropertyusetypegfa', 'neighborhood', 'zipcode', 'latitude', 'longitude'], axis=1)
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3189 entries, 0 to 3375
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   primarypropertytype        3189 non-null   object 
 1   councildistrictcode        3189 non-null   object 
 2   numberofbuildings          3189 non-null   Int64  
 3   numberoffloors             3189 non-null   int64  
 4   propertygfatotal           3189 non-null   int64  
 5   propertygfaparking         3189 non-null   int64  
 6   listofallpropertyusetypes  3189 non-null   object 
 7   largestpropertyusetype     3189 non-null   object 
 8   energystarscore            2386 non-null   object 
 9   siteeuiwn_kbtu_sf          3189 non-null   float64
 10  sourceeuiwn_kbtu_sf        3189 non-null   float64
 11  siteenergyuse_kbtu         3189 non-null   float64
 12  siteenergyusewn_kbtu       3189 non-null   float64
 13  steam                      3189 non-null   objec

In [28]:
# CSV export
df_stats.to_csv('co2_predictions.csv', index=False)
files.download('co2_predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#10.Recommandations

In [29]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.bar(df, x="neighborhood", y="totalghgemissions",
             title="Aménagement des espaces verts",
             color="totalghgemissions",
             color_continuous_scale="greens",
             color_continuous_midpoint=0)

# Add annotation for the conclusion
fig.add_annotation(
    xref='paper', yref='paper',
    x=0.5, # Centered
    y=-0.42,
    text="A la place du centre-ville, privilégier l'Est et la zone industrielle de Duwamish.",
    showarrow=False,
    font=dict(size=20, color='black')
)

fig.update_layout(
    height=900,
    xaxis_title="Neighborhood",
    xaxis=dict(
        title_standoff=30,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        showgrid=False,
        tickfont=dict(size=12),
    ),
    margin=dict(
        t=120,  # Increase the top margin
        b=260   # Increase the bottom margin
    ),
    title=dict(
        x=0.5,
        font=dict(size=30)
    )
)

fig.show()
fig.write_html('neighborhood.html')
files.download('neighborhood.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
import plotly.graph_objects as go
import plotly.express as px

fig = px.scatter(df, x='age', y='totalghgemissions',
                 color_discrete_sequence=['red'],
                 title="Adaptation des politiques de réhabilitation des bâtiments")

# Set the size and symbol based on the 'totalghgemissions' values
fig.update_traces(marker=dict(size=df['totalghgemissions'].abs()*0.012, symbol='circle'))

# Set the x-axis range
fig.update_xaxes(range=[0, 130])

# Set the y-axis range
fig.update_yaxes(range=[-1000, 15000])

# Add annotation for the conclusion
fig.add_annotation(
    xref='paper', yref='paper',
    x=0.5,  # Centered
    y=-0.2,
    text="Subventionner les travaux de rénovation énergétique en fonction de la pollution effective, au lieu de l'âge.",
    showarrow=False,
    font=dict(size=20, color='black')
)

fig.update_layout(
    height=900,
    xaxis_title="Age",
    xaxis=dict(
        title_standoff=30,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        showgrid=False,
        tickfont=dict(size=12),
    ),
    margin=dict(
        t=120,  # Increase the top margin
        b=160   # Increase the bottom margin
    ),
    title=dict(
        x=0.5,
        font=dict(size=30)
    )
)

fig.show()
fig.write_html('age.html')
files.download('age.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
fig = px.scatter(df, x='energystarscore', y='totalghgemissions', color_discrete_sequence=['blue'], title="Utilisation du score ENERGY STAR")

# Set the size and symbol based on the 'totalghgemissions' values
fig.update_traces(marker=dict(size=df['totalghgemissions'].abs()*0.012, symbol='circle'))

# Set the x-axis range
fig.update_xaxes(range=[0, 105])

# Set the y-axis range
fig.update_yaxes(range=[-1000, 14000])

# Add annotation at the bottom
fig.add_annotation(
    xref='paper', yref='paper',
    x=0.5,   # Centered
    y=-0.2,
    text="Accompagner la recherche d'une nouvelle cotation : <a href='https://www.researchgate.net/publication/342831494_EnergyStar_Towards_more_accurate_and_explanatory_building_energy_benchmarking' target='_blank'>EnergyStar++ (Berkeley / National University of Singapore)</a>",
    showarrow=False,
    font=dict(size=20, color='black')
)

fig.update_layout(
    xaxis_type='linear',  # Set x-axis scale to linear (because 'energystarscore' is categorical)
    height=900,
    xaxis_title="ENERGY STAR Score",
    yaxis_title="Total GHG Emissions",
    xaxis=dict(
        title_standoff=30,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        showgrid=False,
        tickfont=dict(size=12),
    ),
    margin=dict(
        t=120,  # Increase the top margin
        b=160   # Increase the bottom margin
    ),
    title=dict(
        x=0.5,
        font=dict(size=30)
    )
)

fig.show()

fig.write_html('estar.html')
files.download('estar.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>